In [8]:
from typing import Any
import numpy as np
from PIL import Image
from OpenGL.GL import *
import glfw


def createTexture(imagePath: str) -> wrapper:
    """
    Create a texture from an image file.

    Args:
        imagePath (str): The file path to the image.

    Returns:
        GLuint: OpenGL texture ID.
    """
    image = Image.open(imagePath)
    image = image.transpose(Image.FLIP_TOP_BOTTOM)
    img_data = np.array(image.convert("RGBA"), np.uint8)

    texture: wrapper = glGenTextures(1)
    glBindTexture(GL_TEXTURE_2D, texture)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, image.width, image.height, 0, GL_RGBA, GL_UNSIGNED_BYTE, img_data)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    glBindTexture(GL_TEXTURE_2D, 0)

    return texture


def drawQuad(texture: Any, x: float, y: float, width: float, height: float) -> None:
    """
    Draw a textured quad at a given position and dimensions.

    Args:
        texture (Any): OpenGL texture ID.
        x (float): X-coordinate of the bottom-left corner of the quad.
        y (float): Y-coordinate of the bottom-left corner of the quad.
        width (float): Width of the quad.
        height (float): Height of the quad.
    """
    glEnable(GL_TEXTURE_2D)
    glBindTexture(GL_TEXTURE_2D, texture)

    glBegin(GL_QUADS)
    glTexCoord2f(0, 0)
    glVertex2f(x, y)
    glTexCoord2f(1, 0)
    glVertex2f(x + width, y)
    glTexCoord2f(1, 1)
    glVertex2f(x + width, y + height)
    glTexCoord2f(0, 1)
    glVertex2f(x, y + height)
    glEnd()

    glBindTexture(GL_TEXTURE_2D, 0)
    glDisable(GL_TEXTURE_2D)


def main() -> None:
    """
    Main function to initialize GLFW and OpenGL, then run the loop.
    """
    if not glfw.init():
        print("GLFW initialization failed")
        return

    window = glfw.create_window(800, 600, "Image Display Example", None, None)
    if not window:
        print("Window creation failed")
        glfw.terminate()
        return

    glfw.make_context_current(window)

    texture = createTexture('test.png')

    x, y = -0.5, -0.5  # Coordinates for the bottom-left corner of the quad
    width, height = 1.0, 1.0  # Dimensions of the quad

    while not glfw.window_should_close(window):
        glClear(GL_COLOR_BUFFER_BIT)

        drawQuad(texture, x, y, width, height)

        glfw.swap_buffers(window)
        glfw.poll_events()

    glfw.terminate()


if __name__ == "__main__":
    main()

In [1]:
from diffusers import DiffusionPipeline
import torch
print(torch.cuda.is_available())


pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16,
                                         use_safetensors=True, variant="fp16")

pipe.to("cuda")

# if using torch < 2.0
# pipe.enable_xformers_memory_efficient_attention()

prompt = "An astronaut riding a green horse"

images = pipe(prompt=prompt).images[0]


C:\Dev\GAIT\venvWin2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


False


WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.0.1+cu118 with CUDA 1108 (you have 2.0.1+cpu)
    Python  3.11.5 (you have 3.11.4)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
Loading pipeline components...: 100%|██████████| 7/7 [00:02<00:00,  2.78it/s]


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
from diffusers import DiffusionPipeline
import torch
import matplotlib.pyplot as plt

# load both base & refiner
base = DiffusionPipeline.from_pretrained(
    "../Models/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
base.to("cuda")
refiner = DiffusionPipeline.from_pretrained(
    "../Models/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
)
refiner.to("cuda")

# Define how many steps and what % of steps to be run on each experts (80/20) here
n_steps = 40
high_noise_frac = 0.8

prompt = "A majestic lion jumping from a big stone at night"

# run both experts
image = base(
    prompt=prompt,
    num_inference_steps=n_steps,
    denoising_end=high_noise_frac,
    output_type="latent",
).images
image = refiner(
    prompt=prompt,
    num_inference_steps=n_steps,
    denoising_start=high_noise_frac,
    image=image,
).images[0]

plt.imshow(image)
